In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

# Load Dataset

In [2]:
df = pd.read_csv("Reviews.csv",index_col=0)

In [3]:
df.head()

,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
Id,,,,,,,,,
1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
df["review"] = df["Score"].apply(lambda x: 0 if x<4 else 1)
df = df[["Text","review"]]

In [5]:
df

,Text,review
Id,,
1,I have bought several of the Vitality canned d...,1
2,Product arrived labeled as Jumbo Salted Peanut...,0
3,This is a confection that has been around a fe...,1
4,If you are looking for the secret ingredient i...,0
5,Great taffy at a great price. There was a wid...,1
...,...,...
568450,Great for sesame chicken..this is a good if no...,1
568451,I'm disappointed with the flavor. The chocolat...,0
568452,"These stars are small, so you can give 10-15 o...",1


In [6]:
df["review"].value_counts() # 1 for positive and 0 for negative

1    443777
0    124677
Name: review, dtype: int64

The dataset has almost 4 times the positive reviews compared to negative reviews. To counter this we can sample only a part of the positive reviews

In [7]:
positive_reviews = df[df.review == 1]
negative_reviews = df[df.review == 0]

positive_reviews = positive_reviews.sample(n=len(negative_reviews))

reviews = positive_reviews.append(negative_reviews).reset_index(drop=True)

In [33]:
reviews
# The data can be shuffled later when calling model.fit()

,Text,review
0,Both my cats adore this brand food in all the ...,1
1,My 80 year old Mom suffered from excruciating ...,1
2,This is the smoothest and best tasting Keurig ...,1
3,These chips have the right amount of crunch an...,1
4,"I am new to Quinoa, but have seen it in bulk i...",1
...,...,...
249349,I just bought this soup today at my local groc...,0
249350,This soup is mostly broth. Although it has a k...,0
249351,"It is mostly broth, with the advertised 3/4 cu...",0
249352,I had ordered some of these a few months back ...,0


In [34]:
sentences = reviews["Text"].values
labels = reviews["review"].values

In [35]:
# Define training and testing sets
train_sentences,test_sentences,train_labels,test_labels = train_test_split(sentences,labels,test_size=0.3,shuffle=True)

# Tokenization

In [36]:
embed_dim = 64 # dimension of the embedding layer
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [38]:
tokenizer = Tokenizer(oov_token=oov_tok) #Define tokenizer
tokenizer.fit_on_texts(train_sentences) # Assign tokens based on words on training set

vocab_size = len(tokenizer.word_index) + 1 # add 1 to account for the '0' used to pad sequences

train_sequences = tokenizer.texts_to_sequences(train_sentences) # Create sequences based on tokens for the training set

max_length = np.max(list(map(lambda x: len(x), train_sequences))) # Extract the length of the longest sentence in the train set

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type) # pad/truncate zeros at the end for a length of 'max_length' 


test_sequences = tokenizer.texts_to_sequences(test_sentences) # similar preprocessing for test set
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Define Model

In [104]:
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embed_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
optimizer = tf.keras.optimizers.Adam()
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 64)           640000    
_________________________________________________________________
bidirectional (Bidirectional (None, 120, 128)          66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 16)                1040      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 748,321
Trainable params: 748,321
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels),batch_size=64)

Epoch 1/10
1555/1555 [==============================] - 62s 40ms/step - loss: 0.2651 - accuracy: 0.8905 - val_loss: 0.2223 - val_accuracy: 0.9076
Epoch 2/10
1555/1555 [==============================] - 61s 39ms/step - loss: 0.1909 - accuracy: 0.9251 - val_loss: 0.1888 - val_accuracy: 0.9263
Epoch 3/10
1555/1555 [==============================] - 61s 39ms/step - loss: 0.1573 - accuracy: 0.9399 - val_loss: 0.1822 - val_accuracy: 0.9308
Epoch 4/10
1555/1555 [==============================] - 61s 39ms/step - loss: 0.1336 - accuracy: 0.9503 - val_loss: 0.1793 - val_accuracy: 0.9321
Epoch 5/10
1555/1555 [==============================] - 61s 39ms/step - loss: 0.1143 - accuracy: 0.9583 - val_loss: 0.1907 - val_accuracy: 0.9342
Epoch 6/10
1555/1555 [==============================] - ETA: 0s - loss: 0.0982 - accuracy: 0.9647